In [1]:
from pyspark.sql import SparkSession
import getpass 
username=getpass.getuser()
spark=SparkSession. \
    builder. \
    config('spark.ui.port','0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    config('spark.shuffle.useOldFetchProtocol', 'true'). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

In [2]:
loans_repay_raw_df = spark.read \
.format("csv") \
.option("header",True) \
.option("inferSchema", True) \
.load("/public/trendytech/lendingclubproject/raw/loans_repayments_csv")

In [3]:
loans_repay_raw_df

loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d
141581221,1055.81,2591.7,0.0,3647.51,709.23,Mar-2019,Apr-2019
141506948,1252.75,306.04,0.0,1558.79,312.63,Mar-2019,Apr-2019
141357400,626.37,354.96,0.0,981.33,197.27,Mar-2019,Apr-2019
139445427,1118.16,297.36,0.0,1415.52,283.95,Mar-2019,Apr-2019
141407409,1169.72,3605.3,0.0,4775.02,964.9,Mar-2019,Apr-2019
141360802,2313.98,2512.88,0.0,4826.86,952.02,Mar-2019,Apr-2019
141163960,4689.63,1994.93,0.0,6684.56,1342.57,Mar-2019,Apr-2019
141533932,585.29,640.53,15.0,1240.82,235.13,Mar-2019,Apr-2019
141441276,2030.82,762.81,0.0,2793.63,477.62,Mar-2019,Apr-2019
141569080,1803.55,1110.59,0.0,2914.14,585.91,Mar-2019,Apr-2019


In [4]:
loans_repay_raw_df.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- total_rec_prncp: string (nullable = true)
 |-- total_rec_int: string (nullable = true)
 |-- total_rec_late_fee: string (nullable = true)
 |-- total_pymnt: string (nullable = true)
 |-- last_pymnt_amnt: string (nullable = true)
 |-- last_pymnt_d: string (nullable = true)
 |-- next_pymnt_d: string (nullable = true)



In [5]:
loans_repay_schema = 'loan_id string, total_principal_received float, total_interest_received float, total_late_fee_received float, total_payment_received float, last_payment_amount float, last_payment_date string, next_payment_date string'

In [6]:
loans_repay_raw_df = spark.read \
.format("csv") \
.option("header",True) \
.schema(loans_repay_schema) \
.load("/public/trendytech/lendingclubproject/raw/loans_repayments_csv")

In [7]:
loans_repay_raw_df.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- total_principal_received: float (nullable = true)
 |-- total_interest_received: float (nullable = true)
 |-- total_late_fee_received: float (nullable = true)
 |-- total_payment_received: float (nullable = true)
 |-- last_payment_amount: float (nullable = true)
 |-- last_payment_date: string (nullable = true)
 |-- next_payment_date: string (nullable = true)



In [8]:
from pyspark.sql.functions import current_timestamp

In [9]:
loans_repay_df_ingestd = loans_repay_raw_df.withColumn("ingest_date", current_timestamp())

In [10]:
loans_repay_df_ingestd

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
141581221,1055.81,2591.7,0.0,3647.51,709.23,Mar-2019,Apr-2019,2023-10-04 05:40:...
141506948,1252.75,306.04,0.0,1558.79,312.63,Mar-2019,Apr-2019,2023-10-04 05:40:...
141357400,626.37,354.96,0.0,981.33,197.27,Mar-2019,Apr-2019,2023-10-04 05:40:...
139445427,1118.16,297.36,0.0,1415.52,283.95,Mar-2019,Apr-2019,2023-10-04 05:40:...
141407409,1169.72,3605.3,0.0,4775.02,964.9,Mar-2019,Apr-2019,2023-10-04 05:40:...
141360802,2313.98,2512.88,0.0,4826.86,952.02,Mar-2019,Apr-2019,2023-10-04 05:40:...
141163960,4689.63,1994.93,0.0,6684.56,1342.57,Mar-2019,Apr-2019,2023-10-04 05:40:...
141533932,585.29,640.53,15.0,1240.82,235.13,Mar-2019,Apr-2019,2023-10-04 05:40:...
141441276,2030.82,762.81,0.0,2793.63,477.62,Mar-2019,Apr-2019,2023-10-04 05:40:...
141569080,1803.55,1110.59,0.0,2914.14,585.91,Mar-2019,Apr-2019,2023-10-04 05:40:...


In [11]:
loans_repay_df_ingestd.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- total_principal_received: float (nullable = true)
 |-- total_interest_received: float (nullable = true)
 |-- total_late_fee_received: float (nullable = true)
 |-- total_payment_received: float (nullable = true)
 |-- last_payment_amount: float (nullable = true)
 |-- last_payment_date: string (nullable = true)
 |-- next_payment_date: string (nullable = true)
 |-- ingest_date: timestamp (nullable = false)



In [12]:
loans_repay_df_ingestd.count()

2260701

In [13]:
loans_repay_df_ingestd.createOrReplaceTempView("loan_repayments")

In [14]:
spark.sql("select count(*) from loan_repayments where total_principal_received is null")

count(1)
69


In [15]:
columns_to_check = ["total_principal_received", "total_interest_received", "total_late_fee_received", "total_payment_received", "last_payment_amount"]

In [16]:
loans_repay_filtered_df = loans_repay_df_ingestd.na.drop(subset=columns_to_check)

In [17]:
loans_repay_filtered_df.count()

2260498

In [18]:
loans_repay_filtered_df.createOrReplaceTempView("loan_repayments")

In [19]:
spark.sql("select count(*) from loan_repayments where total_payment_received = 0.0")

count(1)
995


In [20]:
spark.sql("select count(*) from loan_repayments where total_payment_received = 0.0 and total_principal_received != 0.0")

count(1)
46


In [21]:
spark.sql("select * from loan_repayments where total_payment_received = 0.0 and total_principal_received != 0.0")

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
1064185,11600.98,11600.98,10000.0,0.0,0.0,0.0,Dec-2014,2023-10-04 05:40:...
516382,21890.229,21856.03,16000.0,0.0,0.0,0.0,Mar-2014,2023-10-04 05:40:...
528899,3045.0364,3019.64,2500.0,0.0,0.0,0.0,Jan-2013,2023-10-04 05:40:...
527598,2398.9092,2220.51,2200.0,0.0,0.0,0.0,Jul-2011,2023-10-04 05:40:...
525697,21797.86,19894.9,15750.0,0.0,0.0,0.0,Jun-2015,2023-10-04 05:40:...
522641,3146.8193,3146.82,3000.0,0.0,0.0,0.0,Sep-2011,2023-10-04 05:40:...
515655,29938.576,29905.75,22800.0,0.0,0.0,0.0,May-2013,2023-10-04 05:40:...
501234,15219.313,15155.9,12000.0,0.0,0.0,0.0,May-2013,2023-10-04 05:40:...
498194,11642.714,11031.47,10000.0,0.0,0.0,0.0,Jan-2013,2023-10-04 05:40:...
495171,11138.843,10024.96,10000.0,0.0,0.0,0.0,Apr-2013,2023-10-04 05:40:...


In [22]:
from pyspark.sql.functions import when, col

In [23]:
loans_payments_fixed_df = loans_repay_filtered_df.withColumn(
   "total_payment_received",
    when(
        (col("total_principal_received") != 0.0) &
        (col("total_payment_received") == 0.0),
        col("total_principal_received") + col("total_interest_received") + col("total_late_fee_received")
    ).otherwise(col("total_payment_received"))
)

In [24]:
loans_payments_fixed_df

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
141581221,1055.81,2591.7,0.0,3647.51,709.23,Mar-2019,Apr-2019,2023-10-04 05:40:...
141506948,1252.75,306.04,0.0,1558.79,312.63,Mar-2019,Apr-2019,2023-10-04 05:40:...
141357400,626.37,354.96,0.0,981.33,197.27,Mar-2019,Apr-2019,2023-10-04 05:40:...
139445427,1118.16,297.36,0.0,1415.52,283.95,Mar-2019,Apr-2019,2023-10-04 05:40:...
141407409,1169.72,3605.3,0.0,4775.02,964.9,Mar-2019,Apr-2019,2023-10-04 05:40:...
141360802,2313.98,2512.88,0.0,4826.86,952.02,Mar-2019,Apr-2019,2023-10-04 05:40:...
141163960,4689.63,1994.93,0.0,6684.56,1342.57,Mar-2019,Apr-2019,2023-10-04 05:40:...
141533932,585.29,640.53,15.0,1240.82,235.13,Mar-2019,Apr-2019,2023-10-04 05:40:...
141441276,2030.82,762.81,0.0,2793.63,477.62,Mar-2019,Apr-2019,2023-10-04 05:40:...
141569080,1803.55,1110.59,0.0,2914.14,585.91,Mar-2019,Apr-2019,2023-10-04 05:40:...


In [25]:
loans_payments_fixed_df.filter("loan_id == '1064185'")

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
1064185,11600.98,11600.98,10000.0,33201.96,0.0,0.0,Dec-2014,2023-10-04 05:40:...


In [26]:
loans_payments_fixed2_df = loans_payments_fixed_df.filter("total_payment_received != 0.0")

In [27]:
loans_payments_fixed2_df.filter("last_payment_date = 0.0").count()

48

In [28]:
loans_payments_fixed2_df.filter("next_payment_date = 0.0").count()

24

In [29]:
loans_payments_fixed2_df.filter("last_payment_date is null").count()

1477

In [30]:
loans_payments_fixed2_df.filter("next_payment_date is null").count()

1344240

In [31]:
loans_payments_ldate_fixed_df = loans_payments_fixed2_df.withColumn(
  "last_payment_date",
   when(
       (col("last_payment_date") == 0.0),
       None
       ).otherwise(col("last_payment_date"))
)

In [32]:
loans_payments_ndate_fixed_df = loans_payments_ldate_fixed_df.withColumn(
  "last_payment_date",
   when(
       (col("next_payment_date") == 0.0),
       None
       ).otherwise(col("next_payment_date"))
)

In [33]:
loans_payments_ndate_fixed_df.filter("last_payment_date = 0.0").count()

0

In [34]:
loans_payments_ndate_fixed_df.filter("next_payment_date = 0.0").count()

24

In [35]:
loans_payments_ndate_fixed_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/user/itv006277/lendingclubproject/raw/cleaned/loans_repayments_parquet") \
.save()

In [36]:
loans_payments_ndate_fixed_df.write \
.option("header", True) \
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv006277/lendingclubproject/raw/cleaned/loans_repayments_csv") \
.save()